## model 按token输出

In [2]:
from typing import Any
from vllm import LLM
from transformers import AutoTokenizer, TextStreamer
from langchain_community.llms import VLLM
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
import sys


def on_llm_new_token(self, token: str, **kwargs: Any) -> None:
    sys.stdout.write(token)
    sys.stdout.flush()


model_name = "/root/models/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
streamer = TextStreamer(tokenizer)
llm_llama3 = VLLM(
    model=model_name,
    tokenizer=tokenizer,
    task="text-generation",
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
    temperature=0.2,
    do_sample=True,
    repetition_penalty=1.1,
    return_full_text=False,
    max_new_tokens=900,
)

2024-05-22 16:41:00,384	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-22 16:41:00 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/root/models/Qwen1.5-1.8B-Chat', speculative_config=None, tokenizer='/root/models/Qwen1.5-1.8B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/root/models/Qwen1.5-1.8B-Chat)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-22 16:41:01 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-22 16:41:01 selector.py:27] Using FlashAttention-2 backend.
INFO 05-22 16:41:03 model_runner.py:175] Loading model weights took 3.4653 GB
INFO 05-22 16:41:04 gpu_executor.py:114] # GPU blocks: 5356, # CPU blocks: 1365
INFO 05-22 16:41:06 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-22 16:41:06 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-22 16:41:10 model_runner.py:1017] Graph capturing finished in 4 secs.


In [5]:
from threading import Thread
from transformers import TextIteratorStreamer
inputs = tokenizer(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextIteratorStreamer(tokenizer)

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
# generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=20)
# thread = Thread(target=llm_llama3.generate, kwargs=generation_kwargs)
# thread.start()
# generated_text = ""
# for new_text in streamer:
#     generated_text += new_text
#     print(generated_text)
llm_llama3.generate(["nihao"], streamer=streamer, max_new_tokens=20)

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.96s/it]


LLMResult(generations=[[Generation(text='\n1. 你认为人工智能在医疗领域的应用前景如何？\n2. 你认为人工智能在医疗领域的应用有哪些挑战和限制？\n3. 你认为人工智能在医疗领域的应用有哪些优势和潜力？\n4. 你认为人工智能在医疗领域的应用有哪些局限性？\n5. 你认为人工智能在医疗领域的应用有哪些未来发展趋势？\n6. 你认为人工智能在医疗领域的应用有哪些伦理和法律问题？\n7. 你认为人工智能在医疗领域的应用有哪些社会影响？\n8. 你认为人工智能在医疗领域的应用有哪些经济影响？\n9. 你认为人工智能在医疗领域的应用有哪些技术挑战？\n10. 你认为人工智能在医疗领域的应用有哪些未来挑战？\n\n11. 你认为人工智能在医疗领域的应用有哪些具体案例？\n12. 你认为人工智能在医疗领域的应用有哪些成功的经验和教训？\n13. 你认为人工智能在医疗领域的应用有哪些未解决的问题和挑战？\n14. 你认为人工智能在医疗领域的应用有哪些未来研究方向？\n15. 你认为人工智能在医疗领域的应用有哪些可能的伦理和法律问题？\n16. 你认为人工智能在医疗领域的应用有哪些社会影响？\n17. 你认为人工智能在医疗领域的应用有哪些经济影响？\n18. 你认为人工智能在医疗领域的应用有哪些技术挑战？\n19. 你认为人工智能在医疗领域的应用有哪些未来挑战？\n20. 你认为人工智能在医疗领域的应用有哪些可能的应用场景？')]], llm_output=None, run=[RunInfo(run_id=UUID('a285b50a-6ec9-4d7c-b149-45592bfd19c0'))])

## transfomer streaming test

In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer
from vllm import LLM, SamplingParams

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name)

llm_llama3 = LLM(
    model=model_name,
    tokenizer=model_name,
    # temperature=0.2,
    # do_sample=True,
    # repetition_penalty=1.1,
    # return_full_text=False,
    # max_new_tokens=900,
)

# inputs = tok(["世界你好"], return_tensors="pt")
streamer = TextStreamer(tok)

# Despite returning the usual output, the streamer will also print the generated text to stdout.
# _ = llm_llama3.generate(**inputs, streamer=streamer, max_new_tokens=20)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-23 10:59:18 llm_engine.py:100] Initializing an LLM engine (v0.4.2) with config: model='/root/models/Qwen1.5-1.8B-Chat', speculative_config=None, tokenizer='/root/models/Qwen1.5-1.8B-Chat', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, tokenizer_revision=None, trust_remote_code=False, dtype=torch.bfloat16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), seed=0, served_model_name=/root/models/Qwen1.5-1.8B-Chat)


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


INFO 05-23 10:59:19 utils.py:660] Found nccl from library /root/.config/vllm/nccl/cu12/libnccl.so.2.18.1
INFO 05-23 10:59:20 selector.py:27] Using FlashAttention-2 backend.
INFO 05-23 10:59:21 model_runner.py:175] Loading model weights took 3.4653 GB
INFO 05-23 10:59:22 gpu_executor.py:114] # GPU blocks: 5356, # CPU blocks: 1365
INFO 05-23 10:59:24 model_runner.py:937] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 05-23 10:59:24 model_runner.py:941] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 05-23 10:59:28 model_runner.py:1017] Graph capturing finished in 4 secs.


In [16]:

from vllm import SamplingParams
from transformers import TextStreamer

streamer = TextStreamer(tok)
sampling_params = SamplingParams(top_p=0.95,
                                 temperature=0.2,
                                 #  do_sample=True,
                                 repetition_penalty=1.1,
                                 max_tokens=900,

                                 #  return_full_text=False,
                                 #  max_new_tokens=900,
                                 )


outputs = llm_llama3.generate(
    "其中 LLM类主要运行离线程序、SamplingParams负责设置调用模型当中的一些参数", sampling_params=sampling_params)

# llm_llama3("其中 LLM类主要运行离线程序、SamplingParams负责设置调用模型当中的一些参数")

for output in outputs:
    prompt = output.prompt
    generated_text = output.outputs[0].text
    print(f"Prompt: {prompt!r}, Generated text: {generated_text!r}")

Processed prompts: 100%|██████████| 1/1 [00:01<00:00,  1.38s/it]

Prompt: '其中 LLM类主要运行离线程序、SamplingParams负责设置调用模型当中的一些参数', Generated text: '，如batch_size、num_epochs等，以及在训练过程中监控模型的性能指标，如loss、accuracy等。例如：\n\n```python\nclass SamplingParams:\n    def __init__(self, batch_size=32, num_epochs=10):\n        self.batch_size = batch_size\n        self.num_epochs = num_epochs\n\n    def sample(self, inputs, targets):\n        # 这里是用于生成样本数据的代码，根据输入和目标进行随机抽样\n        pass\n```\n\n在这个例子中，`sample`方法接受两个参数：`inputs`和`targets`，分别表示输入和目标数据。它会根据这些数据生成一个样本数据集，并返回这个样本数据集。\n\n需要注意的是，LSTM类中的`sample`方法需要依赖于具体的实现细节，比如使用的数据结构（如numpy数组或张量）、随机数生成算法（如numpy.random.choice或sklearn.choice）以及具体的训练策略（如梯'


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextStreamer

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, temperature=0.2,
                                             do_sample=True,
                                             repetition_penalty=1.1,)
inputs = tok(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextStreamer(tok)

# Despite returning the usual output, the streamer will also print the generated text to stdout.
_ = model.generate(**inputs, streamer=streamer,
                   max_new_tokens=200)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextIteratorStreamer
from threading import Thread

model_name = "/root/models/Qwen1.5-1.8B-Chat"
tok = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(llm_llama3)

inputs = tok(["An increasing sequence: one,"], return_tensors="pt")
streamer = TextIteratorStreamer(tok)

# Run the generation in a separate thread, so that we can fetch the generated text in a non-blocking way.
generation_kwargs = dict(inputs, streamer=streamer, max_new_tokens=20)
thread = Thread(target=model.generate, kwargs=generation_kwargs)
thread.start()
generated_text = ""
for new_text in streamer:
    generated_text += new_text
    print(generated_text)